In [1]:
import pickle
import pandas as pd

In [5]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import tensorwatch

In [61]:
class BasicModule(nn.Module):
    def __init__(self):
        super(BasicModule, self).__init__()
        self.model_name = str(type(self))

    def load(self, path):
        self.load_state_dict(torch.load(path))

    def save(self, path):
        torch.save(self.state_dict(), path)

In [3]:
read_train = open("../dataset/train.pkl", "rb")
df_train = pickle.load(read_train)
read_train.close()

In [128]:
read_test = open("../dataset/test.pkl", "rb")
df_test = pickle.load(read_test)
read_test.close()

In [10]:
feat_columns = [2] + list(range(5,26))

In [11]:
[df_train.columns[i] for i in feat_columns]

['Wide target sequence\n(Total 47 bps = 4 bp neighboring sequence + 20 bp protospacer + 3 bp NGG PAM+ 20 bp neighboring sequence)',
 'PBS length',
 'RT length',
 'PBS-RT length',
 "Prime edited sequence\n(input for deep learning, A/C/G/T indicates 3' extension (RT template-PBS) binding region)",
 'Tm1\n(PBS)',
 'Tm 2\n(target DNA region corresponding to RT template)',
 'Tm 3\n(reverse transcribed cDNA and PAM-opposite DNA strand)',
 'Tm 4\n(RT template region and reverse transcribed cDNA)',
 'deltaTm\n(Tm3-Tm2)',
 'GC count_1\n(PBS)',
 'GC count_2\n(RT)',
 'GC count_3\n(PBS-RT)',
 'GC contents_1\n(PBS)',
 'GC contents_2\n(RT)',
 'GC contents_3\n(PBS-RT)',
 'MFE_1\n(pegRNA)',
 'MFE_2\n(-spacer)',
 'MFE_3\n(RT-PBS-PolyT)',
 'MFE_4\n(spacer only)',
 'MFE_5\n(Spacer+Scaffold)',
 'DeepSpCas9 score']

In [171]:
def encode_dna_seq(seq):
    encoding = np.zeros((len(seq), 4))
    # encode dna seq
    for i in range(len(seq)):
        # one hot encoding
        pos = "acgt".find(seq[i].lower())
        if pos >= 0:
            encoding[i][pos] += 1
    return encoding

def encode_dna_data(data):
    data_len = len(data)
    seq_len = len(data[0])
    data_encoding = None
    # one hot encoding
    embed = torch.nn.Embedding.from_pretrained(torch.FloatTensor([[0,0,0,0],
                                                                 [1,0,0,0],
                                                                 [0,1,0,0],
                                                                 [0,0,1,0],
                                                                 [0,0,0,1]    
                                                                 ]))
    baseToIndex ={
        "n":0,
        "x":0,
        "a":1,
        "c":2,
        "g":3,
        "t":4
    }
    
    
    return embed(torch.IntTensor([[baseToIndex[b.lower()] for b in data[I]] for I in range(data_len)]))

In [166]:
embed = torch.nn.Embedding.from_pretrained(torch.FloatTensor([[0,0,0,0],
                                                             [1,0,0,0],
                                                             [0,1,0,0],
                                                             [0,0,1,0],
                                                             [0,0,0,1]    
                                                             ]))
embed(torch.LongTensor([[0,2,1], [3,2,1]]))

tensor([[[0., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.]],

        [[0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.]]])

In [173]:
encode_dna_data(df_train[df_train.columns[8]])[19]

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0.,

In [156]:
list(df_train[df_train.columns[8]])

['...',
 ...]

In [186]:
del sdf
# sdf = PEDataset(df_train)

In [192]:
class PEDataset(Dataset):
     
    def __init__(self, dataframe):
        super(PEDataset, self).__init__()
        target_col = 2
        pegrna_col = 8
        feat_cols = [5,6,7] + list(range(9,26))
        y_col = 26
        
        self.target_seq = encode_dna_data(dataframe[dataframe.columns[target_col]].values)
        self.pegrna_seq = encode_dna_data(dataframe[dataframe.columns[pegrna_col]].values)
        self.y = torch.FloatTensor(dataframe[dataframe.columns[y_col]].values)
        self.feat_list = torch.FloatTensor(dataframe[dataframe.columns[feat_cols]].values)
            
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return (self.target_seq[index], self.pegrna_seq[index], self.feat_list[index], self.y[index])
        
    # return a subset of dataset of given range
#     def get_subset(self, start, end):
        
#         return DNADataset([(self.x[i],self.y[i]) for i in range(start, end)], self.size)

In [193]:
train_set = PEDataset(df_train)
# with open("../dataset/train_dataset.pkl", "wb") as train_save:
#     pickle.dump(train_set, train_save)



In [195]:
test_set = PEDataset(df_test)
# with open("../dataset/test_dataset.pkl", "wb") as test_save:
#     pickle.dump(test_set, test_save)

In [231]:
class SimpleLSTM(BasicModule):
    def __init__(self,args):
        super(SimpleLSTM, self).__init__()
        self.dropout = nn.Dropout(args['dropout'])
        self.params = args
        self.model_type = "LSTM"
        self.lstm_target = nn.LSTM(input_size=args['num_features'],
                                   hidden_size=args['hidden_size'], 
                                   num_layers=args['num_layers'],
                                   batch_first=True,
                                   dropout=args['dropout'], 
                                   bidirectional=args['bidirectional'])
        
        self.lstm_pegrna = nn.LSTM(input_size=args['num_features'], 
                                   hidden_size=args['hidden_size'], 
                                   num_layers=args['num_layers'],
                                   batch_first=True, 
                                   dropout=args['dropout'],
                                   bidirectional=args['bidirectional'])
        
        N = 2 if self.params['bidirectional'] else 1
        self.fc1 = nn.Linear(args['hidden_size'] * (args['seq_length'] //2) * 2 * N + args['num_extra_feat'], args['out1'], bias=True)
        self.fc2 = nn.Linear(args['out1'], args['out2'], bias=True)
        self.bn = nn.BatchNorm1d(args['out1'])
        self.relu = nn.ReLU(inplace=True)
#         self.hidden = None
        
#     def init_hidden(self, batch_size):
#         N = 2 if self.params['bidirectional'] else 1
#         hidden = (Variable(torch.zeros(self.params['num_layers']*N, batch_size, self.params['hidden_size'])),
#                   Variable(torch.zeros(self.params['num_layers']*N, batch_size, self.params['hidden_size'])))
#         self.hidden = hidden
        
    
    def forward(self, target, pegrna, feat):
        batch_size = target.size(0)
        lstm_target_out = self.lstm_target(target)[0]
        lstm_pegrna_out = self.lstm_pegrna(pegrna)[0]
#         print(lstm_target_out.shape, lstm_pegrna_out.shape)
        out_target = F.max_pool2d(lstm_target_out, (2,1), stride=(2,1))
        out_pegrna = F.max_pool2d(lstm_pegrna_out, (2,1), stride=(2,1))
#         print(out_target.shape, out_pegrna.shape)
        out_target = out_target.view(batch_size, -1)
        out_pegrna = out_pegrna.view(batch_size, -1)
        x = torch.cat((out_target, out_pegrna, feat), 1) 
#         print(x.shape)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.bn(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        
        return x
    
    def setDropout(self, dropout):
        self.dropout.p = dropout

In [232]:
test_set.y

tensor([ 0.0000e+00,  6.7718e-02, -2.5720e-02,  ...,  2.7540e+01,
         1.4465e+01,  1.7867e-01])

In [241]:
lstm_args = {
    'seq_length': 47,
    'bidirectional': True,
    'num_features': 4,
    'num_layers': 2,
    'hidden_size': 40,
    'dropout': 0.3,
    'out1': 90,
    'out2': 1,
    'num_extra_feat': 20
}
lstm_model = SimpleLSTM(lstm_args)


In [242]:
def train(model, train_dataset, epochs, batch_size, val_set, cv=0.1, learning_rate=0.001):
    subset = train_dataset
    n_training_samples = len(subset) * (1-cv)
    n_val_samples = len(subset) * cv
    train_loader =torch.utils.data.DataLoader(subset, batch_size=batch_size,
                                              num_workers=0)
#     val_loader =torch.utils.data.DataLoader(subset, batch_size=100,
#                                               sampler=SubsetRandomSampler(
#                                                   np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64)
#                                               ), num_workers=0)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()


    print("Train %s samples."%n_training_samples)
    
    for e in range(epochs):
        # train loss
        epoch_train_loss = 0.0
        model.setDropout(model.params['dropout'])
        for t, p, f, scores in train_loader:
#             inp = Variable(inp)
            out = model(t,p,f).squeeze()
            scores = Variable(torch.FloatTensor(scores))
            loss = criterion(out, scores)
            epoch_train_loss += loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(epoch_train_loss)
                
        # validation loss
#         epoch_val_loss = 0.0
#         for inp, labels in val_loader:
#             inp = inp.unsqueeze(1)
#             out = model(inp)        
#             loss = criterion(out, labels)
#             epoch_val_loss += loss


#         print(str(epoch_train_loss) + ' ')
#         print(str(epoch_val_loss.tolist()) + '\n')
        model.setDropout(0.0)
        print(e)
        tscores = test_scores(model, val_set, 400)
        corr = pd.DataFrame([[float(val_set[i][-1]), tscores[i]] for i in range(len(tscores))], columns=['endogenous', 'integrated']).corr(method='spearman')['endogenous']['integrated']
        print(corr)
        if corr >= 0.80:
            print("Saving.")
            model.save("../model/%s_2L_40H_BI_90FC_%s.model"%(model.model_type, round(corr, 3)))

In [243]:
train(lstm_model, train_set, 15 , 120, val_set=test_set, cv=0.0, learning_rate=0.003)

Train 38692.0 samples.
tensor(21435.6699, grad_fn=<AddBackward0>)
0
0.6967340632665414
tensor(14464.1318, grad_fn=<AddBackward0>)
1
0.7241464130906617
tensor(13156.9121, grad_fn=<AddBackward0>)
2
0.7427520314556039
tensor(12244.5371, grad_fn=<AddBackward0>)
3
0.7557826697062178
tensor(11425.3809, grad_fn=<AddBackward0>)
4
0.7482166303725658
tensor(10766.6074, grad_fn=<AddBackward0>)
5
0.7571389843991534
tensor(9921.1074, grad_fn=<AddBackward0>)
6
0.7624253473610509
tensor(9109.2441, grad_fn=<AddBackward0>)
7
0.7656431471841116
tensor(8681.1113, grad_fn=<AddBackward0>)
8
0.7577586723162014
tensor(8248.4561, grad_fn=<AddBackward0>)
9
0.7631072536226189


KeyboardInterrupt: 

In [139]:
train(lstm_model, train_set, 5 , 120, train_set, cv=0.0, learning_rate=0.003, start=0, end=12832)

Train 38692.0 samples.
tensor(7701.7275, grad_fn=<AddBackward0>)
tensor(7421.0303, grad_fn=<AddBackward0>)
tensor(6968.8027, grad_fn=<AddBackward0>)
tensor(6803.5527, grad_fn=<AddBackward0>)
tensor(6458.0591, grad_fn=<AddBackward0>)


In [210]:
#
def test_scores(model, test_set, batch_size):
    tscores = []
    for t, p, f, scores in DataLoader(test_set, batch_size=120):
        model.zero_grad()
        out = model(t,p,f).squeeze()
        for v in out.detach().numpy():
            tscores.append(v)
    return tscores

In [211]:
tscores = test_scores(lstm_model, test_set, 400)
corr = pd.DataFrame([[test_set[i][2], float(test_set[i][-1]), tscores[i]] for i in range(len(tscores))], columns=['seq', 'endogenous', 'integrated'])\
       [['endogenous', 'integrated']].corr(method='spearman')
corr

,endogenous,integrated
endogenous,1.000000,0.780733
integrated,0.780733,1.000000


In [203]:
tscores

[0.095474824,
 0.112517945,
 0.11169062,
 0.12633686,
 0.11576359,
 0.119406916,
 0.11079997,
 0.13203523,
 0.083242014,
 0.110848024,
 0.08969596,
 0.116763934,
 0.08505379,
 0.09844707,
 0.090627216,
 0.13470948,
 0.13234888,
 0.124309525,
 0.093577735,
 0.09589605,
 0.0920716,
 0.08956043,
 0.078727365,
 0.12169789,
 0.12250391,
 0.11109529,
 0.0690963,
 0.11320336,
 0.09971639,
 0.11101929,
 0.11668636,
 0.097446546,
 0.1199781,
 0.10320313,
 0.090665,
 0.11591448,
 0.12103957,
 0.12639609,
 0.12476415,
 0.117662266,
 0.11521844,
 0.11581837,
 0.10913208,
 0.11954358,
 0.07768363,
 0.13761866,
 0.11017036,
 0.083371565,
 0.11304813,
 0.10795714,
 0.12019821,
 0.106128186,
 0.09303429,
 0.1169747,
 0.1338746,
 0.120289735,
 0.111375146,
 0.12438977,
 0.0963246,
 0.13080622,
 0.10794156,
 0.1124381,
 0.09601771,
 0.13235232,
 0.09802013,
 0.096344404,
 0.08138539,
 0.090183005,
 0.08805727,
 0.09677868,
 0.09657556,
 0.12298192,
 0.13582031,
 0.09967448,
 0.105614476,
 0.09741069,
 0